## **LightGBM (Time series Regression)**

In [ ]:
#import library
import numpy as np
import pandas as pd
import lightgbm as lgb
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.width', None)

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day07.git

In [ ]:
# Data import
train = pd.read_csv("/content/LG_time_series_day07/Data_LightGBM_train.csv", parse_dates=['date'])
test = pd.read_csv("/content/LG_time_series_day07/Data_LightGBM_test.csv", parse_dates=['date'])
df = pd.concat([train, test], sort=False)
df

In [ ]:
# sales variable = target variable
# for checking the information of data
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(train)

In [ ]:
# IQR method 사용: Q1:0.05%, Q3:0.95%
# for outlier check
def outlier_thresholds(dataframe, col_name, q1_perc=0.05, q3_perc=0.95):
    """
    given dataframe, column name, q1_percentage and q3 percentage, function calculates low_limit and up_limit

    """
    quartile1 = dataframe[col_name].quantile(q1_perc)
    quartile3 = dataframe[col_name].quantile(q3_perc)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def check_outlier(dataframe, col_name, q1_perc=0.01, q3_perc=0.99):
    outlier_list = []
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1_perc=0.01, q3_perc=0.99)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True

    else:
        return False

check_outlier(df, 'sales')

In [ ]:
print(f"total number of stores: {df['store'].nunique()}\n")
print(f"total number of items: {df['item'].nunique()}\n")
print(f"total number of items per each store: {df.groupby(['store'])['item'].nunique()}")

In [ ]:
# target 변수 sales의 sum, mean ,median, std 계산
df.groupby(["store", "item"]).agg({"sales": ["sum", "mean", "median", "std"]})

In [ ]:
# time series decomposition
train_plot = train.set_index('date')
y = train_plot['sales'].resample('MS').mean() 

result = sm.tsa.seasonal_decompose(y, model='additive')
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(8, 6)

In [ ]:
# feature engineering
# seanolity 찾기 위해서 date variable로 새로운 feature 형성
def create_date_features(df):
    df['month'] = df.date.dt.month
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    df['week_of_year'] = df.date.dt.weekofyear
    # 1.1.2013 is Tuesday, so our starting point is the 2nd day of week
    df['day_of_week'] = df.date.dt.dayofweek + 1
    df['year'] = df.date.dt.year
    df["is_wknd"] = df.date.dt.weekday // 4
    df['is_month_start'] = df.date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.date.dt.is_month_end.astype(int)
    return df

In [ ]:
df = create_date_features(df)
df.head()

In [ ]:
# Random noise
# overfitting 방지 위해 random noise(gaussian random noise) 추가
def random_noise(dataframe):

    return np.random.normal(size=(len(dataframe),))

In [ ]:
# Lag & Shifted Features
# sort the values per store, item and date so that values would be shifted equally
df.sort_values(by=['store', 'item', 'date'], axis=0, inplace=True)

def lag_features(dataframe, lags):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

df = lag_features(df, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])

In [ ]:
df

In [ ]:
# Rolling mean & Moving Average
def roll_mean_features(dataframe, windows):
    dataframe = dataframe.copy()
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store", "item"])['sales']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(dataframe)
    return dataframe

df = roll_mean_features(df, [365, 546, 730])

In [ ]:
df

In [ ]:
# Exponentially weighted mean features
def ewm_features(dataframe, alphas, lags):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store", "item"])['sales']. \
                    transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]

df = ewm_features(df, alphas, lags)

In [ ]:
df

In [ ]:
# one-hot encoding을 통해 dummy 변수 생성
df = pd.get_dummies(df, columns=['store', 'item', 'day_of_week', 'month'])

df['sales'] = np.log1p(df["sales"].values)

In [ ]:
# SMAPE (Symmetric Mean Absolute Percentage Error) 평가지표 사용
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

In [ ]:
# train 데이터에는 2013-2016에 해당하는 데이터가 포함됨
train = df.loc[(df["date"] < "2017-01-01"), :]
train["date"].min(), train["date"].max()

In [ ]:
# val 데이터에는 2017년 1월 1일-2017년 4월 1일에 해당하는 데이터가 포함됨
val = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :]

In [ ]:
# 불필요한 columns 삭제 
cols = [col for col in train.columns if col not in ['date', 'id', "sales", "year"]]

In [ ]:
# 최종 train, validation data
Y_train = train['sales']
X_train = train[cols]

Y_val = val['sales']
X_val = val[cols]

In [ ]:
#light gbm parameter
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 15000,
              'early_stopping_rounds': 200,
              'nthread': -1}

In [ ]:
# dataset load
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

In [ ]:
# model train
model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=200)

In [ ]:
# validation data에 있는 sales 값이 예측되며 이는 log value로 계산됨
y_pred_val = model.predict(X_val)

In [ ]:
# log value에 지수 취해준 후 smape 평가지표 계산
smape(np.expm1(y_pred_val), np.expm1(Y_val))

In [ ]:
# 변수중요도 계산
lgb.plot_importance(model, max_num_features=20, figsize=(10, 10), importance_type="gain")
plt.show()

In [ ]:
# train과 validation data가 concatenate된 후 train, test data(sales(타겟변수) NAN으로 표시됨) 생성
train = df.loc[~df.sales.isna()]
Y_train = train['sales']
X_train = train[cols]
test = df.loc[df.sales.isna()]
X_test = test[cols]

In [ ]:
# model train시 도출된 best iteration 값 대입
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}

In [ ]:
# 최적의 파라미터로 model train 후 test data prediction 도출
lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)
test_preds = final_model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
# 최종 결과: 2018년 1월 1일 부터 2018년 3월 31일까지의 sales를 의미함
submission_df = test.loc[:, ['id', 'sales']]
submission_df['sales'] = np.expm1(test_preds)
submission_df['id'] = submission_df.id.astype(int)

submission_df.head()